In [9]:
# Import datasets

import pandas as pd
import numpy as np
from pathlib import Path
import get_reversions as gr

# Define the path

path_dataset = Path().cwd().parent / 'oii-fsds-wikipedia-main' / 'DataFrames'

# Import both datasets

greta = pd.read_feather(path_dataset / 'Greta Thunberg.feather')
naomi = pd.read_feather(path_dataset / 'Naomi Klein.feather')

# Add a new column to each DataFrame to indicate the source
greta['environmentalist'] = 'Greta Thunberg'
naomi['environmentalist'] = 'Naomi Klein'

# Merge both
compared = pd.concat([greta, naomi], axis=0, ignore_index=True)


In [11]:
reversion = gr.get_revisions(compared)

/Users/chicokerche/Library/CloudStorage/OneDrive-Pessoal/10_Oxford/Classes/MT/Fundamental_python/week_2/goup-wikipedia-revisions-project/analysis/get_reversions.py:13: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  reversions = df[df['comment'].str.lower().str.contains('(revert(ed)?|undid)', regex=True, na=False)].copy()
